In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [211]:
import os
from glob import glob
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import tensorflow as tf

In [99]:
train_data = '/kaggle/input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/train'

In [161]:
# Load and preprocess the images
def load_and_preprocess_images(file_paths, target_size=(224, 224)):
    images = [cv2.resize(cv2.imread(file_path), target_size) for file_path in file_paths]
    images = np.array(images) / 255.0  # Normalize pixel values to the range [0, 1]
    return images
# Load labeled data with resizing to (100, 100)
X_labeled = load_and_preprocess_images(labeled_files, target_size=(100, 100))
batch_images_resized = tf.image.resize(batch_images, (100, 100))


In [158]:
# Read and show train images
train_images = []
train_labels = []
class_mapping = {'benign': 0, 'malignant': 1}

In [176]:
for folder in os.listdir(train_data):
    data = gb.glob(os.path.join(train_data, folder, '*.jpg'))
    print(f'{len(data)} in folder {folder}')
    
    # Read train images
    for file_path in data:
        image = cv2.imread(file_path)
        image_array = cv2.resize(image, (100, 100))
        train_images.append(image_array)
        train_labels.append(class_mapping[folder])

# Convert data to arrays
X_labeled = np.array(train_images)
y_labeled = np.array(train_labels)

# Use LabelEncoder for encoding labels
label_encoder = LabelEncoder()
y_labeled_encoded = label_encoder.fit_transform(y_labeled)

print("X_labeled shape:", X_labeled.shape)
print("y_labeled_encoded shape:", y_labeled_encoded.shape)

5000 in folder benign
4605 in folder malignant
X_labeled shape: (19210, 100, 100, 3)
y_labeled_encoded shape: (19210,)


In [164]:
# Load and preprocess the images with resizing to (224, 224)
def load_and_preprocess_images(file_paths, target_size=(100, 100)):
    images = [cv2.resize(cv2.imread(file_path), target_size) for file_path in file_paths]
    images = np.array(images) / 255.0  # Normalize pixel values to the range [0, 1]
    return images

# Step 1: Load labeled data
labeled_files = glob(os.path.join(train_data, '*.jpg'))

# Load labeled data with resizing
X_labeled = load_and_preprocess_images(labeled_files)

# Use pseudo-labels as labels for training
label_encoder = LabelEncoder()
y_labeled_encoded = label_encoder.fit_transform([os.path.basename(file) for file in labeled_files])

In [177]:
# Step 3: Load MobileNetV2 as a pre-trained backbone
# Create the MobileNetV2 model with input shape (224, 224, 3)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the weights of the pre-trained model



In [178]:
# Step 4: Create a custom contrastive head
embedding_size = 64
contrastive_head = models.Sequential([
    layers.GlobalAveragePooling2D(),
    layers.Dense(embedding_size, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(embedding_size, activation='relu')
])

In [204]:
# Step 5: Build the full model with both the backbone and the custom head
input_labeled = layers.Input(shape=(224, 224, 3))
output_labeled = contrastive_head(base_model(input_labeled))
num_classes = 2
flat = layers.Flatten()(base_model.output)
output = layers.Dense(num_classes, activation='softmax')(flat)

full_model = models.Model(inputs=input_labeled, outputs=output_labeled)

In [205]:
# Step 6: Compile the model with an appropriate optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
full_model.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [206]:
print("X_labeled shape:", X_labeled.shape)
print("y_labeled_encoded shape:", y_labeled_encoded.shape)


X_labeled shape: (19210, 100, 100, 3)
y_labeled_encoded shape: (19210,)


In [208]:
# Define the train_step function
@tf.function
def train_step(images, labels):
    images_resized = tf.image.resize(images, (224, 224))  # Resize images to match model input shape
    with tf.GradientTape() as tape:
        # Forward pass
        logits = full_model(images_resized, training=True)
        # Compute the loss value using categorical crossentropy
        loss = tf.keras.losses.categorical_crossentropy(labels, logits)

    # Compute gradients
    gradients = tape.gradient(loss, full_model.trainable_variables)
    # Update weights
    optimizer.apply_gradients(zip(gradients, full_model.trainable_variables))

    print(loss)


In [209]:
# Step 9: Train the model on labeled data
epochs = 10
batch_size = 32

# Convert labels to one-hot encoding for categorical crossentropy
y_labeled_one_hot = tf.keras.utils.to_categorical(y_labeled_encoded, num_classes=num_classes)

# Training loop
for epoch in range(epochs):
    epoch_loss = 0.0
    for step in range(0, len(X_labeled), batch_size):
        start_idx = step
        end_idx = step + batch_size

        batch_images = X_labeled[start_idx:end_idx]
        batch_labels = y_labeled_one_hot[start_idx:end_idx]

        # Train on the batch
        train_step(batch_images, batch_labels)

    # Optionally, calculate and print the average loss for the epoch
    average_loss = epoch_loss / (len(X_labeled) // batch_size)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}')


ValueError: in user code:

    File "/tmp/ipykernel_125/352571257.py", line 9, in train_step  *
        loss = tf.keras.losses.categorical_crossentropy(labels, logits)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/losses.py", line 2221, in categorical_crossentropy
        return backend.categorical_crossentropy(
    File "/opt/conda/lib/python3.10/site-packages/keras/src/backend.py", line 5573, in categorical_crossentropy
        target.shape.assert_is_compatible_with(output.shape)

    ValueError: Shapes (32, 2) and (32, 64) are incompatible
